In [180]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
diabet = pd.read_csv('data/diabetes_data.csv')
print(diabet.shape[0])
diabet.head()

778


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [181]:
mask = diabet.duplicated()
diabet_dup = diabet[mask]
print(diabet_dup.shape[0])
diabet = diabet.drop_duplicates()
diabet.shape[0]

10


768

In [182]:
low_information_cols = [] 

for col in diabet.columns:
    top_freq = diabet[col].value_counts(normalize=True).max()
    max_uniq = diabet[col].nunique() / diabet[col].count()
    if top_freq > 0.95:
        low_information_cols.append(col)
    
    if max_uniq > 0.95:
        low_information_cols.append(col)
diabet.drop(low_information_cols, inplace=True, axis=1)
    

In [183]:
cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in cols:
    diabet[col] = diabet[col].apply(lambda x: np.nan if x==0 else x)
diabet['Insulin'].isnull().mean()

0.4869791666666667

In [184]:
tr = diabet.shape[0] * 0.7
diabet = diabet.dropna(axis=1, thresh=tr)
diabet.shape[1]

8

In [185]:
diabet = diabet.dropna(axis=0, thresh=6)
diabet.shape[0]

761

In [186]:
diabet = diabet.fillna(value=diabet.median())
diabet['SkinThickness'].mean()
# for cols in diabet.index:
#     diabet = diabet[col].fillna(value=diabet[col].median())
# diabet['SkinThickness'].mean()


29.109067017082786

In [187]:

def iqr_metod(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    q_25 = data[feature].quantile(0.25)
    q_75 = data[feature].quantile(0.75)
    iqr = q_75 - q_25
    left_bound = q_25 - 1.5*iqr
    right_bound = q_75 + 1.5*iqr
    outliers = data[(x<left_bound)|(x>right_bound)]
    cleaned = data[(x>left_bound)&(x<right_bound)]
    return outliers, cleaned
def z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    sigma = data[feature].std()
    mean = data[feature].mean()
    left_bound = mean - 3*sigma
    right_bound = mean + 3*sigma
    out = data[(x<left_bound)|(x>right_bound)]
    clean = data[(x>left_bound)&(x<right_bound)]
    return out, clean
out, cln = iqr_metod(diabet, 'SkinThickness')
print(f'Метод межкварт. размаха: {out.shape[0]}')
ou, cl = z_score(diabet, 'SkinThickness')
print(f'Метод трёх сигм: {ou.shape[0]}')



Метод межкварт. размаха: 87
Метод трёх сигм: 4


In [245]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
def iqr_metod(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    q_25 = x.quantile(0.25)
    q_75 = x.quantile(0.75)
    iqr = q_75 - q_25
    left_bound = q_25 - 1.5*iqr
    right_bound = q_75 + 1.5*iqr
    outliers = data[(x<left_bound)|(x>right_bound)]
    cleaned = data[(x>left_bound)&(x<right_bound)]
    return outliers, cleaned

out_iqr, cln_iqr = iqr_metod(diabet, 'DiabetesPedigreeFunction')
out_iqr_mode, cln_iqr_mode = iqr_metod(diabet,'DiabetesPedigreeFunction', log_scale=True)
out_iqr.shape[0] - out_iqr_mode.shape[0]


29